# Diffusion Model Backtesting on GPU

This notebook runs your diffusion trading strategy backtest on Google Colab's free GPU.

**Steps:**
1. Upload your trained model
2. Upload market data CSV
3. Run backtest with GPU acceleration
4. Download results

## 1. Setup Environment

In [ ]:
# Clone your project (or upload as zip)
!git clone https://github.com/manupareekk/diffusiontrading.git
%cd diffusiontrading

# Or if uploading manually:
# from google.colab import files
# uploaded = files.upload()  # Upload your project zip
# !unzip diffusion-trading.zip
# %cd diffusiontrading

In [ ]:
# Install dependencies
!pip install -q torch pandas numpy plotly streamlit

In [ ]:
# Verify GPU is available
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Upload Files

In [ ]:
from google.colab import files
import pickle

# Upload trained model (you'll export this from your Mac)
print("Upload your trained model (model.pt):")
uploaded_model = files.upload()

# Upload market data CSV
print("\nUpload your market data CSV:")
uploaded_data = files.upload()

## 3. Load Model and Data

In [ ]:
import sys
sys.path.insert(0, '.')

import pandas as pd
import torch
from src.models.diffusion.ddpm import DDPM
from src.models.networks.temporal_conv import TemporalConvDenoiser
from src.models.schedulers.noise_schedule import create_noise_schedule

# Load model
checkpoint = torch.load('model.pt', map_location='cuda' if torch.cuda.is_available() else 'cpu')
model = checkpoint['model']
model_config = checkpoint['config']

print(f"Model loaded: {model_config}")

# Load data
data_filename = list(uploaded_data.keys())[0]
data = pd.read_csv(data_filename, index_col=0, parse_dates=True)
print(f"\nData loaded: {len(data)} rows")
print(data.head())

## 4. Run Backtest

In [ ]:
from src.backtesting.diffusion_backtest import run_diffusion_backtest, BacktestConfig
from tqdm.notebook import tqdm

# Configure backtest
config = BacktestConfig(
    initial_capital=100000,
    commission=0.001,
    slippage_bps=5.0,
    fast_mode=False,  # Run on full data with GPU
)

# Progress callback
pbar = None
def progress_callback(progress, message):
    global pbar
    if pbar is None:
        pbar = tqdm(total=100, desc="Backtesting")
    pbar.n = int(progress * 100)
    pbar.set_postfix_str(message)
    pbar.refresh()

# Run backtest
result = run_diffusion_backtest(
    data=data,
    model=model,
    model_config=model_config,
    strategy_type="Diffusion - Bands",
    config=config,
    band_type="percentile",
    band_width="percentile_5_95",
    num_samples=100,  # GPU can handle this!
    progress_callback=progress_callback,
)

if pbar:
    pbar.close()

print("\n✅ Backtest Complete!")

## 5. View Results

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Print metrics
print("Performance Metrics:")
print("=" * 50)
for key, value in result.metrics.items():
    if isinstance(value, float):
        print(f"{key:20s}: {value:.4f}")
    else:
        print(f"{key:20s}: {value}")

# Plot equity curve
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                   vertical_spacing=0.05, row_heights=[0.7, 0.3])

equity = result.equity_curve

fig.add_trace(go.Scatter(
    x=equity.index,
    y=equity.values,
    mode='lines',
    name='Equity',
    line=dict(color='blue', width=1.5)
), row=1, col=1)

# Drawdown
running_max = equity.cummax()
drawdown = (equity - running_max) / running_max * 100

fig.add_trace(go.Scatter(
    x=drawdown.index,
    y=drawdown.values,
    mode='lines',
    fill='tozeroy',
    name='Drawdown',
    line=dict(color='red', width=1)
), row=2, col=1)

fig.update_layout(
    height=600,
    showlegend=True,
    title="Backtest Results",
    yaxis_title="Portfolio Value ($)",
    yaxis2_title="Drawdown (%)"
)

fig.show()

## 6. Download Results

In [ ]:
# Save results to CSV
equity_df = result.equity_curve.to_frame('equity')
equity_df.to_csv('backtest_equity.csv')

trades_df = pd.DataFrame(result.trades)
trades_df.to_csv('backtest_trades.csv', index=False)

# Download
files.download('backtest_equity.csv')
files.download('backtest_trades.csv')

print("✅ Results downloaded!")